In [ ]:
# !pip install -r ../requirements.txt

In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

In [2]:
NBAC_DIR = "../../data/fire/nbac"
SAMPLE_SIZE = 10

In [2]:
FUEL_COUNT_FILE = "../data/processed/5_fuel_type_fire_analysis.csv"

In [3]:
nbac_fire_files = sorted(os.listdir(NBAC_DIR))
nbac_fire_files

['nbac_2005_20240530.zip',
 'nbac_2006_20240530.zip',
 'nbac_2007_20240530.zip',
 'nbac_2008_20240530.zip',
 'nbac_2009_20240530.zip',
 'nbac_2010_20240530.zip',
 'nbac_2011_20240530.zip',
 'nbac_2012_20240530.zip',
 'nbac_2013_20240530.zip',
 'nbac_2014_20240530.zip',
 'nbac_2015_20240530.zip',
 'nbac_2016_20240530.zip',
 'nbac_2017_20240530.zip',
 'nbac_2018_20240530.zip',
 'nbac_2019_20240530.zip',
 'nbac_2020_20240530.zip',
 'nbac_2021_20240530.zip',
 'nbac_2022_20240530.zip',
 'nbac_2023_20240530.zip']

In [4]:
nbac_file = lambda file: f"{NBAC_DIR}{os.sep}{file}"

In [5]:
file_path = "../../data/forest/fuel_type/FBP_fueltypes_Canada_30m_EPSG3978_20240522.tif"

In [6]:
# Metadata from "https://cwfis.cfs.nrcan.gc.ca/downloads/nbac/nbac_1972_2023_20240530_shp_metadata.pdf" Pages 22-23
province_label = {
    "AB": "Alberta",
    "BC": "British Columbia",
    "MB": "Manitoba",
    "NB": "New Brunswick",
    "NL": "Newfoundland and Labrador",
    "NS": "Nova Scotia",
    "NT": "Northwest Territories",
    "NU": "Nunavut",
    "ON": "Ontario",
    "PC": "Parks Canada",
    "PE": "Prince Edward Island",
    "QC": "Quebec",
    "SK": "Saskatchewan",
    "YT": "Yukon Territory",
}

In [215]:
color_label = [
    [-9999, (255, 255, 255, 255), ""],
    [1, (209, 255, 115, 255), "C-1 Spruce-Lichen Woodland"],
    [2, (34, 102, 51, 255), "C-2 Boreal Spruce"],
    [3, (131, 199, 149, 255), "C-3 Mature Jack or Lodgepole Pine"],
    [4, (112, 168, 0, 255), "C-4 Immature Jack or Lodgepole Pine"],
    [5, (223, 184, 230, 255), "C-5 Red and White Pine"],
    [7, (112, 12, 242, 255), "C-7 Ponderosa Pine / Douglas Fir"],
    [11, (196, 189, 151, 255), "D-1 Leafless Aspen"],
    [13, (196, 189, 151, 255), "D-1/D-2 Aspen"],
    [31, (255, 255, 190, 255), "O-1a Matted Grass"],
    [101, (130, 130, 130, 255), "Non-fuel"],
    [102, (115, 223, 255, 255), "Water"],
    [105, (204, 204, 204, 255), "Vegetated Non-Fuel"],
    [415, (255, 211, 127, 255), "M-1 Boreal Mixedwood - Leafless (15% Conifer)"],
    [625, (255, 211, 127, 255), "M-1/M-2 Boreal Mixedwood (25% Conifer)"],
    [650, (255, 211, 127, 255), "M-1/M-2 Boreal Mixedwood (50% Conifer)"],
    [675, (255, 211, 127, 255), "M-1/M-2 Boreal Mixedwood (75% Conifer)"],
]

# Convert RGBA values (0-255) to Matplotlib format (0-1)
color_dict = {entry[0]: np.array(entry[1]) / 255 for entry in color_label}
label_color_dict = {entry[2]: np.array(entry[1]) / 255 for entry in color_label}
label_dict = {entry[0]: entry[2] for entry in color_label}

# Create colormap & normalizer
unique_values = list(color_dict.keys())
colors = [color_dict[val] for val in unique_values]
cmap = mcolors.ListedColormap(colors)
norm = mcolors.BoundaryNorm(unique_values + [max(unique_values) + 1], cmap.N)

In [8]:
def filter_to_fire_region(
    image_data, 
    no_data_val:int = -9999
):
    filtered_image = image_data[0][image_data[0] != no_data_val]
    del image_data
    return filtered_image

In [9]:
def get_fuel_type_counts(
    filtered_data
):
    fuel_types, counts = np.unique(
        filtered_data, 
        return_counts=True
    )
    count_dict = {
        int(fuel_type): int(count) 
        for fuel_type, count in zip(fuel_types, counts)
    }
    del fuel_types
    del counts
    return count_dict

In [10]:
def get_year(
    file_name:str
):
    return file_name.split("_")[1]

In [89]:
def get_labels(
    fuel_types:list,
    all_labels:dict = label_dict
):
    return [all_labels[fuel_id] for fuel_id in fuel_types]

def get_colors(
    fuel_types:list,
    all_colors:dict = color_dict
):
    return [all_colors[fuel_id] for fuel_id in fuel_types]

In [12]:
yearly_fuel_counts = {}
with rasterio.open(file_path) as dataset:
    print(f"CRS: {dataset.crs}")

    for fire_index, fire_file in enumerate(nbac_fire_files):
        yearly_fuel_counts[fire_file] = {}

        # get fire data for one year
        fire_year_gpd = gpd.read_file(nbac_file(fire_file))
        fire_year_gpd.to_crs(dataset.crs)
        assert fire_year_gpd.crs == dataset.crs, f"CRS is not same!!!"

        # get data for admin areas
        admin_areas = fire_year_gpd['ADMIN_AREA'].unique()

        # init progress bar
        progress_bar = tqdm(
            iterable = admin_areas,      # The list to loop over
            desc = fire_file,            # The descreption in tqdm
            position = fire_index,       # This is the position of tqdm
            leave = True                          
        ) 

        for admin_area in admin_areas:

            # updated progress bar details
            progress_bar.set_postfix_str(admin_area)

            fire_geom_admin_area = fire_year_gpd[fire_year_gpd['ADMIN_AREA'] == admin_area]['geometry']

            # Mask the raster using the fire polygons
            out_image, out_transform = mask(
                dataset, 
                fire_geom_admin_area, 
                crop=True
            )

            fire_pixels = filter_to_fire_region(
                image_data = out_image
            )
            del out_image
            fuel_counts = get_fuel_type_counts(
                filtered_data = fire_pixels
            )

            yearly_fuel_counts[fire_file][admin_area] = fuel_counts
            
            del fuel_counts
            progress_bar.update(1)
        
    del dataset



CRS: EPSG:3978


nbac_2005_20240530.zip:   0%|          | 0/12 [00:00<?, ?it/s]

nbac_2006_20240530.zip:   0%|          | 0/12 [00:00<?, ?it/s]

nbac_2007_20240530.zip:   0%|          | 0/13 [00:00<?, ?it/s]

nbac_2008_20240530.zip:   0%|          | 0/13 [00:00<?, ?it/s]

...

nbac_2019_20240530.zip:   0%|          | 0/10 [00:00<?, ?it/s]

nbac_2020_20240530.zip:   0%|          | 0/12 [00:00<?, ?it/s]

nbac_2021_20240530.zip:   0%|          | 0/11 [00:00<?, ?it/s]

nbac_2022_20240530.zip:   0%|          | 0/12 [00:00<?, ?it/s]

nbac_2023_20240530.zip:   0%|          | 0/13 [00:00<?, ?it/s]

In [90]:
fuel_counts_data = []
fuel_type_unique = set()


# Get the unique values
for yearly_fire_shapefile in yearly_fuel_counts:
    for provience in yearly_fuel_counts[yearly_fire_shapefile]:
        fuel_type_unique.update(
            yearly_fuel_counts[yearly_fire_shapefile][provience].keys()
        )
        # print(yearly_fuel_counts[yearly_fire_shapefile][provience])
        # print(yearly_fire_shapefile, provience)

fuel_type_unique = sorted(list(fuel_type_unique))

# generate the data array
for yearly_fire_shapefile in yearly_fuel_counts:
    for provience in yearly_fuel_counts[yearly_fire_shapefile]:
        data_row = [
            get_year(file_name = yearly_fire_shapefile),
            province_label[provience]
        ]
        for fule_type in fuel_type_unique:
            if fule_type in yearly_fuel_counts[yearly_fire_shapefile][provience]:
                data_row.append(
                    yearly_fuel_counts[yearly_fire_shapefile][provience][fule_type]
                )
            else:
                data_row.append(
                    0
                )
        fuel_counts_data.append(data_row)
        del data_row

# build the data frame
fule_count_labels = get_labels(fuel_type_unique)
fule_count_df = pd.DataFrame(
    data = fuel_counts_data,
    columns = ['year', 'provience'] + fule_count_labels
)
fule_count_df


,year,provience,C-1 Spruce-Lichen Woodland,C-2 Boreal Spruce,C-3 Mature Jack or Lodgepole Pine,C-4 Immature Jack or Lodgepole Pine,C-5 Red and White Pine,C-7 Ponderosa Pine / Douglas Fir,D-1 Leafless Aspen,D-1/D-2 Aspen,O-1a Matted Grass,Non-fuel,Water,Vegetated Non-Fuel,M-1 Boreal Mixedwood - Leafless (15% Conifer),M-1/M-2 Boreal Mixedwood (25% Conifer),M-1/M-2 Boreal Mixedwood (50% Conifer),M-1/M-2 Boreal Mixedwood (75% Conifer)
0,2005,Quebec,4269,550576,26629,582399,0,0,103953,4383163,228278,30486,223829,192836,1321289,1086512,97836,426692
1,2005,Ontario,6764,73318,9347,140847,3,0,21904,145824,18917,5359,3524,71,1246,36248,33225,32059
2,2005,Northwest Territories,909,69580,3958,90934,0,0,128892,898309,192408,12811,8955,60188,3626,229673,79509,80272
3,2005,British Columbia,11980,21318,14521,50035,2101,64,4108,48750,9893,398,274,57406,1744,37853,4191,9648
4,2005,Saskatchewan,0,128824,33116,1110402,0,0,108493,227506,81289,870,30824,86699,943,83549,62658,60825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2023,Yukon Territory,0,0,0,0,0,0,0,0,0,0,0,3483955,0,0,0,0
224,2023,Nova Scotia,0,0,0,0,0,0,0,0,0,0,0,253307,0,0,0,0
225,2023,Newfoundland and Labrador,0,0,0,0,0,0,0,0,0,0,0,203905,0,0,0,0
226,2023,Nunavut,0,0,0,0,0,0,0,0,0,0,0,45173,0,0,0,0


In [91]:
fule_count_df.to_csv(
    FUEL_COUNT_FILE,
    index = False
)

In [3]:
fule_count_df = pd.read_csv(
    FUEL_COUNT_FILE,
)
fule_count_df

,year,provience,C-1 Spruce-Lichen Woodland,C-2 Boreal Spruce,C-3 Mature Jack or Lodgepole Pine,C-4 Immature Jack or Lodgepole Pine,C-5 Red and White Pine,C-7 Ponderosa Pine / Douglas Fir,D-1 Leafless Aspen,D-1/D-2 Aspen,O-1a Matted Grass,Non-fuel,Water,Vegetated Non-Fuel,M-1 Boreal Mixedwood - Leafless (15% Conifer),M-1/M-2 Boreal Mixedwood (25% Conifer),M-1/M-2 Boreal Mixedwood (50% Conifer),M-1/M-2 Boreal Mixedwood (75% Conifer)
0,2005,Quebec,4269,550576,26629,582399,0,0,103953,4383163,228278,30486,223829,192836,1321289,1086512,97836,426692
1,2005,Ontario,6764,73318,9347,140847,3,0,21904,145824,18917,5359,3524,71,1246,36248,33225,32059
2,2005,Northwest Territories,909,69580,3958,90934,0,0,128892,898309,192408,12811,8955,60188,3626,229673,79509,80272
3,2005,British Columbia,11980,21318,14521,50035,2101,64,4108,48750,9893,398,274,57406,1744,37853,4191,9648
4,2005,Saskatchewan,0,128824,33116,1110402,0,0,108493,227506,81289,870,30824,86699,943,83549,62658,60825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2023,Yukon Territory,0,0,0,0,0,0,0,0,0,0,0,3483955,0,0,0,0
224,2023,Nova Scotia,0,0,0,0,0,0,0,0,0,0,0,253307,0,0,0,0
225,2023,Newfoundland and Labrador,0,0,0,0,0,0,0,0,0,0,0,203905,0,0,0,0
226,2023,Nunavut,0,0,0,0,0,0,0,0,0,0,0,45173,0,0,0,0
